In [ ]:
import sys,os

sys.path.append("..")
import modeling

import tensorflow as tf
bert_config_file = "/data/xuht/chinese_L-12_H-768_A-12/bert_config.json"
init_checkpoint = "/data/xuht/chinese_L-12_H-768_A-12/bert_model.ckpt"
bert_config = modeling.BertConfig.from_json_file(bert_config_file)
graph = tf.Graph()

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

with graph.as_default():
    sess = tf.Session()
    
    def get_masked_lm_output(bert_config, input_tensor, output_weights, positions,
                         label_ids, label_weights):
        """Get loss and log probs for the masked LM."""
        print(input_tensor.get_shape(), "==before gather indexs==")
        input_tensor = gather_indexes(input_tensor, positions)
        print(positions.get_shape(), "========position shape=======")
        print(input_tensor.get_shape(), label_ids.get_shape())
        with tf.variable_scope("cls/predictions"):
            # We apply one more non-linear transformation before the output layer.
            # This matrix is not used after pre-training.
            with tf.variable_scope("transform"):
                input_tensor = tf.layers.dense(
                  input_tensor,
                  units=bert_config.hidden_size,
                  activation=modeling.get_activation(bert_config.hidden_act),
                  kernel_initializer=modeling.create_initializer(
                      bert_config.initializer_range))
                input_tensor = modeling.layer_norm(input_tensor)

                # The output weights are the same as the input embeddings, but there is
                # an output-only bias for each token.
                output_bias = tf.get_variable(
                    "output_bias",
                    shape=[bert_config.vocab_size],
                    initializer=tf.zeros_initializer())
                logits = tf.matmul(input_tensor, output_weights, transpose_b=True)
                logits = tf.nn.bias_add(logits, output_bias)
                log_probs = tf.nn.log_softmax(logits, axis=-1)

                label_ids = tf.reshape(label_ids, [-1])
                label_weights = tf.reshape(label_weights, [-1])

                one_hot_labels = tf.one_hot(
                    label_ids, depth=bert_config.vocab_size, dtype=tf.float32)

                # The `positions` tensor might be zero-padded (if the sequence is too
                # short to have the maximum number of predictions). The `label_weights`
                # tensor has a value of 1.0 for every real prediction and 0.0 for the
                # padding predictions.
                per_example_loss = -tf.reduce_sum(log_probs * one_hot_labels, axis=[-1])
                numerator = tf.reduce_sum(label_weights * per_example_loss)
                denominator = tf.reduce_sum(label_weights) + 1e-5
                loss = numerator / denominator

                return (loss, per_example_loss, log_probs)


    def get_next_sentence_output(bert_config, input_tensor, labels):
        """Get loss and log probs for the next sentence prediction."""

        # Simple binary classification. Note that 0 is "next sentence" and 1 is
        # "random sentence". This weight matrix is not used after pre-training.
        with tf.variable_scope("cls/seq_relationship"):
            output_weights = tf.get_variable(
                "output_weights",
                shape=[2, bert_config.hidden_size],
                initializer=modeling.create_initializer(bert_config.initializer_range))
            output_bias = tf.get_variable(
                "output_bias", shape=[2], initializer=tf.zeros_initializer())

            logits = tf.matmul(input_tensor, output_weights, transpose_b=True)
            logits = tf.nn.bias_add(logits, output_bias)
            log_probs = tf.nn.log_softmax(logits, axis=-1)
            labels = tf.reshape(labels, [-1])
            one_hot_labels = tf.one_hot(labels, depth=2, dtype=tf.float32)
            per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
            loss = tf.reduce_mean(per_example_loss)
            return (loss, per_example_loss, log_probs)

    def gather_indexes(sequence_tensor, positions):
        """Gathers the vectors at the specific positions over a minibatch."""
        sequence_shape = modeling.get_shape_list(sequence_tensor, expected_rank=3)
        batch_size = sequence_shape[0]
        seq_length = sequence_shape[1]
        width = sequence_shape[2]

        flat_offsets = tf.reshape(
          tf.range(0, batch_size, dtype=tf.int32) * seq_length, [-1, 1])
        flat_positions = tf.reshape(positions + flat_offsets, [-1])
        flat_sequence_tensor = tf.reshape(sequence_tensor,
                                        [batch_size * seq_length, width])
        output_tensor = tf.gather(flat_sequence_tensor, flat_positions)
        return output_tensor
    
    def model_fn(features):
        
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        masked_lm_positions = features["masked_lm_positions"]
        masked_lm_ids = features["masked_lm_ids"]
        masked_lm_weights = features["masked_lm_weights"]
        next_sentence_labels = features["next_sentence_labels"]
    
        model = modeling.BertModel(
              config=bert_config,
              is_training=True,
              input_ids=input_ids,
              input_mask=input_mask,
              token_type_ids=segment_ids,
              use_one_hot_embeddings=False)
    
        (masked_lm_loss,
         masked_lm_example_loss, masked_lm_log_probs) = get_masked_lm_output(
             bert_config, model.get_sequence_output(), model.get_embedding_table(),
             masked_lm_positions, masked_lm_ids, masked_lm_weights)

        (next_sentence_loss, next_sentence_example_loss,
         next_sentence_log_probs) = get_next_sentence_output(
             bert_config, model.get_pooled_output(), next_sentence_labels)

        total_loss = masked_lm_loss + next_sentence_loss
    
        tvars = tf.trainable_variables()
        (assignment_map,
           initialized_variable_names) = modeling.get_assigment_map_from_checkpoint(
           tvars, init_checkpoint)
    
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
#         for var in tvars:
#             init_string = ""
#             if var.name in initialized_variable_names:
#                 init_string = ", *INIT_FROM_CKPT*"
#             tf.logging.info(" name = %s, shape = %s%s", var.name, var.shape,
#                                             init_string)
        return total_loss
        
    filename = "/data/xuht/mrc_search/pretrain/pretrain_debug/chunk_0.tfrecords"
    max_seq_length = 384
    max_predictions_per_seq = 5
    name_to_features = {
        "input_ids":
            tf.FixedLenFeature([max_seq_length], tf.int64),
        "input_mask":
            tf.FixedLenFeature([max_seq_length], tf.int64),
        "segment_ids":
            tf.FixedLenFeature([max_seq_length], tf.int64),
        "masked_lm_positions":
            tf.FixedLenFeature([max_predictions_per_seq], tf.int64),
        "masked_lm_ids":
            tf.FixedLenFeature([max_predictions_per_seq], tf.int64),
        "masked_lm_weights":
            tf.FixedLenFeature([max_predictions_per_seq], tf.float32),
        "next_sentence_labels":
            tf.FixedLenFeature([1], tf.int64),
    }
    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)
        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example
    def tfrecord_train_input_fn():
        tfrecord_dataset = tf.data.TFRecordDataset(filename)
        tfrecord_dataset = tfrecord_dataset.map(lambda   x:_decode_record(x, name_to_features)).shuffle(True).batch(2)
        tfrecord_dataset = tfrecord_dataset.repeat(1)
        tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()

        return tfrecord_iterator.get_next()

    a = tfrecord_train_input_fn()
    total_loss = model_fn(a)
    
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    
    l = sess.run(a)
    
    while True:
        # 不断的获得下一个样本
        try:
            p = sess.run(a)
            break
        except tf.errors.OutOfRangeError:
            print("End of dataset")
            break
        else:
            print(p)
            

In [ ]:
import pickle
pickle.dump(p, open("/data/xuht/bert_test.pkl", 'wb'))

In [1]:
import numpy as np
import tensorflow as tf
import pickle
p = pickle.load(open("/data/xuht/bert_test.pkl", 'rb'))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
import sys,os

sys.path.append("..")
import modeling

import tensorflow as tf
bert_config_file = "/data/xuht/chinese_L-12_H-768_A-12/bert_config.json"
init_checkpoint = "/data/xuht/chinese_L-12_H-768_A-12/bert_model.ckpt"
bert_config = modeling.BertConfig.from_json_file(bert_config_file)
graph = tf.Graph()

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

with graph.as_default():
    sess = tf.Session()


    import numpy as np
    sequence_output = tf.constant(np.random.random((2, 384, 768)).astype(np.float32))
    embedding_table = tf.constant(np.random.random((21128, 768)).astype(np.float32))

    def gather_indexes(sequence_tensor, positions):
            """Gathers the vectors at the specific positions over a minibatch."""
            sequence_shape = modeling.get_shape_list(sequence_tensor, expected_rank=3)
            batch_size = sequence_shape[0]
            seq_length = sequence_shape[1]
            width = sequence_shape[2]

            flat_offsets = tf.reshape(
              tf.range(0, batch_size, dtype=tf.int32) * seq_length, [-1, 1])
            flat_positions = tf.reshape(positions + flat_offsets, [-1])
            flat_sequence_tensor = tf.reshape(sequence_tensor,
                                            [batch_size * seq_length, width])
            output_tensor = tf.gather(flat_sequence_tensor, flat_positions)
            return output_tensor

    def get_masked_lm_output(bert_config, input_tensor, output_weights, positions,
                             label_ids, label_weights):
            """Get loss and log probs for the masked LM."""
            print(input_tensor.get_shape(), "==before gather indexs==")
            input_tensor = gather_indexes(input_tensor, positions)
            print(positions.get_shape(), "========position shape=======")
            print(input_tensor.get_shape(), label_ids.get_shape())
            with tf.variable_scope("cls/predictions"):
                # We apply one more non-linear transformation before the output layer.
                # This matrix is not used after pre-training.
                with tf.variable_scope("transform"):
                    input_tensor = tf.layers.dense(
                      input_tensor,
                      units=bert_config.hidden_size,
                      activation=modeling.get_activation(bert_config.hidden_act),
                      kernel_initializer=modeling.create_initializer(
                          bert_config.initializer_range))
                    input_tensor = modeling.layer_norm(input_tensor)

                    # The output weights are the same as the input embeddings, but there is
                    # an output-only bias for each token.
                    output_bias = tf.get_variable(
                        "output_bias",
                        shape=[bert_config.vocab_size],
                        initializer=tf.zeros_initializer())
                    logits = tf.matmul(input_tensor, output_weights, transpose_b=True)
                    logits = tf.nn.bias_add(logits, output_bias)
                    log_probs = tf.nn.log_softmax(logits, axis=-1)

                    label_ids = tf.reshape(label_ids, [-1])
                    label_weights = tf.reshape(label_weights, [-1])

                    print(label_ids.get_shape(), '==label id shape==')
                    print(label_weights.get_shape(), '==label_weights shape==')

                    one_hot_labels = tf.one_hot(
                        label_ids, depth=bert_config.vocab_size, dtype=tf.float32)

                    # The `positions` tensor might be zero-padded (if the sequence is too
                    # short to have the maximum number of predictions). The `label_weights`
                    # tensor has a value of 1.0 for every real prediction and 0.0 for the
                    # padding predictions.
#                     per_example_loss = -tf.reduce_sum(log_probs * one_hot_labels, axis=[-1])
                    per_example_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
                                                labels=tf.stop_gradient(label_ids),
                                                logits=logits)
                    print(per_example_loss.get_shape(), logits.get_shape())
                    numerator = tf.reduce_sum(label_weights * per_example_loss)
                    denominator = tf.reduce_sum(label_weights) + 1e-5
                    loss = numerator / denominator

                    return (loss, per_example_loss, log_probs)

    (masked_lm_loss,
             masked_lm_example_loss, masked_lm_log_probs) = get_masked_lm_output(
                 bert_config, sequence_output, embedding_table,
                 tf.constant(p['masked_lm_positions']), 
                tf.constant(p['masked_lm_ids']), tf.constant(p['masked_lm_weights']))
    sess.run(tf.global_variables_initializer())
    result = sess.run([masked_lm_loss, masked_lm_example_loss])

(2, 384, 768) ==before gather indexs==
(2, 5) ========position shape=======
(10, 768) (2, 5)
(10,) ==label id shape==
(10,) ==label_weights shape==
(10,) (10, 21128)


In [5]:
result

[30.362099,
 array([28.56466 , 26.971487, 31.349976, 29.427927, 21.093653, 26.532446,
        42.859726, 32.961357, 19.235996, 35.355614], dtype=float32)]

In [9]:
result

[39.846348,
 array([46.08673 , 60.06067 , 17.415873, 55.034992, 48.53436 , 38.077618,
        39.98493 , 38.667385, 22.477818, 40.81152 ], dtype=float32)]

In [11]:
p['masked_lm_weights'].shape

(2, 5)

In [13]:
for key in p:
    print(p[key][0], key)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0] segment_ids
[ 839 6756 3173 6756    0] masked_lm_ids
[72 73 83 84  0] masked_lm_positions
[1. 1. 1. 1. 0.] masked_lm_weights
[0] next_sentence_labels
[ 101 1071 2141 5650  679 2533 4638 6821 4905  679 3221 2769 1762 4692